In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


try:

    df = pd.read_csv('avocado.csv')

    print("--- Carga de Datos Exitosa ---")
    

    print("\n--- 2.1. Primeras filas (df.head()) ---")
    print(df.head())

    
    print("\n--- 2.2. Información del DataFrame (df.info()) ---")
    df.info()

    # Obtenemos estadísticas descriptivas de las variables numéricas
    print("\n--- 2.3. Estadísticas Descriptivas (df.describe()) ---")
    print(df.describe())

except FileNotFoundError:
    print("Error: No se encontró el archivo 'avocado.csv'.")
    print("Por favor, asegúrese de que el archivo esté en el mismo directorio que este script.")

